Carregando o dataset

In [2]:
import os
import pandas as pd
import numpy as np

dataset = pd.read_pickle(r'C:\Users\mathe\OneDrive\Área de Trabalho\SoccerIA\MathIA_v7\dataset_141cols_europeu.pkl')

nan_counts = dataset.isna().sum()
nan_tot = nan_counts.sum()
print(nan_tot)
print(dataset.shape)

0
(66840, 141)


Importando os dataframes de 2023

In [3]:
import os
import pandas as pd
import numpy as np

# Caminho para a pasta que contém os arquivos .xlsx de 2023
path = r'C:\Users\mathe\OneDrive\Área de Trabalho\SoccerIA\Planilhas2023'

# Dicionário para armazenar os dataframes
dataframes = {}
dfss = []
# Lista todos os arquivos na pasta
files = os.listdir(path)

# Filtra a lista de arquivos para incluir apenas os arquivos .xlsx
xlsx_files = [f for f in files if f.endswith('.xlsx')]

# Carrega cada arquivo .xlsx em um dataframe e armazena no dicionário
for file in xlsx_files:
    full_path = os.path.join(path, file)  # junta o caminho do diretório com o nome do arquivo
    dataframes[file] = pd.read_excel(full_path)  # lê o arquivo .xlsx do caminho completo

    if "Unnamed: 0" in dataframes[file].columns:
        dataframes[file].drop("Unnamed: 0", axis=1, inplace=True)

    if "match_report_url" in dataframes[file].columns:
        dataframes[file].drop("match_report_url", axis=1, inplace=True)    

    if "team1_yellow_cards" in dataframes[file].columns:
        dataframes[file].drop("team1_yellow_cards", axis=1, inplace=True)

    if "team2_yellow_cards" in dataframes[file].columns:
        dataframes[file].drop("team2_yellow_cards", axis=1, inplace=True)  
    dfss.append(dataframes[file])

    print(file)
    print(dataframes[file].shape)
    nan_counts = dataframes[file].isna().sum()
    print(nan_counts)


# Combine all the DataFrames into a single DataFrame
combined_df_2023 = pd.concat(dfss, ignore_index=True)


combined_df_2023['team1'] = combined_df_2023['team1'].str.lower()
combined_df_2023['team2'] = combined_df_2023['team2'].str.lower()
combined_df_2023.replace('', np.nan, inplace=True)
# Add a column to mark future matches
combined_df_2023['is_future_match'] = combined_df_2023['team1_goals'].isna() | combined_df_2023['team2_goals'].isna()
combined_df_2023['season'] = '2023'
# Replace empty string with NaN
combined_df_2023["team1_red_cards"].replace('', np.nan, inplace=True)
combined_df_2023["team2_red_cards"].replace('', np.nan, inplace=True)

# Replace NaN with 0
combined_df_2023["team1_red_cards"].fillna(0, inplace=True)
combined_df_2023["team2_red_cards"].fillna(0, inplace=True)

combined_df_2023['date'] = pd.to_datetime(combined_df_2023['date'], format='%Y-%m-%d', errors='coerce')
combined_df_2023.sort_values('date', inplace=True)

print(combined_df_2023.columns)
print(combined_df_2023.shape) 


2023_ALE_B.xlsx
(90, 19)
team1                     0
team2                     0
team1_goals              45
team2_goals              45
season                    0
championship              0
team1_shots_on_target    45
team1_shots_out          45
team2_shots_on_target    45
team2_shots_out          45
team1_red_cards          45
team2_red_cards          45
team1_fouls              45
team2_fouls              45
team1_corners            45
team2_corners            45
team1_total_shots        45
team2_total_shots        45
date                      0
dtype: int64
2023_ALE_C.xlsx
(120, 19)
team1                     0
team2                     0
team1_goals              70
team2_goals              70
season                    0
championship              0
team1_shots_on_target    70
team1_shots_out          70
team2_shots_on_target    70
team2_shots_out          70
team1_red_cards          70
team2_red_cards          70
team1_fouls              70
team2_fouls              70
team1_corner

In [4]:
import pandas as pd
import numpy as np


combined_df_2023.sort_values('date', inplace=True)


combined_df_2023['team1_goals'] = pd.to_numeric(combined_df_2023['team1_goals'], errors='coerce')
combined_df_2023['team2_goals'] = pd.to_numeric(combined_df_2023['team2_goals'], errors='coerce')

# calculate goal differences
combined_df_2023['goal_diff_team1'] = combined_df_2023['team1_goals'] - combined_df_2023['team2_goals']
combined_df_2023['goal_diff_team2'] = combined_df_2023['team2_goals'] - combined_df_2023['team1_goals']

# calculate corners differences
combined_df_2023['corners_diff_team1'] = combined_df_2023['team1_corners'] - combined_df_2023['team2_corners']############# NEW
combined_df_2023['corners_diff_team2'] = combined_df_2023['team2_corners'] - combined_df_2023['team1_corners']############# NEW

# calculate big wins and losses
combined_df_2023['team1_big_win'] = np.where(combined_df_2023['goal_diff_team1'] >= 2, 1, 0)
combined_df_2023['team1_big_loss'] = np.where(combined_df_2023['goal_diff_team1'] <= -2, 1, 0)
combined_df_2023['team2_big_win'] = np.where(combined_df_2023['goal_diff_team2'] >= 2, 1, 0)
combined_df_2023['team2_big_loss'] = np.where(combined_df_2023['goal_diff_team2'] <= -2, 1, 0)

# calculate AH-2.5 win and losses
combined_df_2023['team1_ah-2.5_win'] = np.where(combined_df_2023['corners_diff_team1'] >= 3, 1, 0)############# NEW
combined_df_2023['team1_ah-2.5_loss'] = np.where(combined_df_2023['corners_diff_team1'] <= 2, 1, 0)############# NEW
combined_df_2023['team2_ah-2.5_win'] = np.where(combined_df_2023['corners_diff_team2'] >= 3, 1, 0)############# NEW
combined_df_2023['team2_ah-2.5_loss'] = np.where(combined_df_2023['corners_diff_team2'] <= 2, 1, 0)############# NEW


# calculate AH+2.5 win and losses
combined_df_2023['team1_ah+2.5_win'] = np.where(combined_df_2023['corners_diff_team1'] >= -2, 1, 0)############# NEW
combined_df_2023['team1_ah+2.5_loss'] = np.where(combined_df_2023['corners_diff_team1'] <= -3, 1, 0)############# NEW
combined_df_2023['team2_ah+2.5_win'] = np.where(combined_df_2023['corners_diff_team2'] >= -2, 1, 0)############# NEW
combined_df_2023['team2_ah+2.5_loss'] = np.where(combined_df_2023['corners_diff_team2'] <= -3, 1, 0)############# NEW


# calculate over4.5 win and losses
combined_df_2023['team1_over4.5'] = np.where(combined_df_2023['team1_corners'] >= 5, 1, 0)############# NEW
combined_df_2023['team1_under4.5'] = np.where(combined_df_2023['team1_corners'] <= 4, 1, 0)############# NEW
combined_df_2023['team2_over4.5'] = np.where(combined_df_2023['team2_corners'] >= 5, 1, 0)############# NEW
combined_df_2023['team2_under4.5'] = np.where(combined_df_2023['team2_corners'] <= 4, 1, 0)############# NEW

# calculate over3.5 win and losses
combined_df_2023['team1_over3.5'] = np.where(combined_df_2023['team1_corners'] >= 4, 1, 0)############# NEW
combined_df_2023['team1_under3.5'] = np.where(combined_df_2023['team1_corners'] <= 3, 1, 0)############# NEW
combined_df_2023['team2_over3.5'] = np.where(combined_df_2023['team2_corners'] >= 4, 1, 0)############# NEW
combined_df_2023['team2_under3.5'] = np.where(combined_df_2023['team2_corners'] <= 3, 1, 0)############# NEW

# calculate over6.5 win and losses
combined_df_2023['team1_over6.5'] = np.where(combined_df_2023['team1_corners'] >= 7, 1, 0)############# NEW
combined_df_2023['team1_under6.5'] = np.where(combined_df_2023['team1_corners'] <= 6, 1, 0)############# NEW
combined_df_2023['team2_over6.5'] = np.where(combined_df_2023['team2_corners'] >= 7, 1, 0)############# NEW
combined_df_2023['team2_under6.5'] = np.where(combined_df_2023['team2_corners'] <= 6, 1, 0)############# NEW


# Initialize these columns with 0
combined_df_2023['team1_big_wins_last5'] = 0
combined_df_2023['team1_big_losses_last5'] = 0
combined_df_2023['team2_big_wins_last5'] = 0
combined_df_2023['team2_big_losses_last5'] = 0

combined_df_2023['team1_ah-2.5_wins_last5'] = 0############# NEW
combined_df_2023['team1_ah-2.5_losses_last5'] = 0############# NEW
combined_df_2023['team2_ah-2.5_wins_last5'] = 0############# NEW
combined_df_2023['team2_ah-2.5_losses_last5'] = 0############# NEW

combined_df_2023['team1_ah+2.5_wins_last5'] = 0############# NEW
combined_df_2023['team1_ah+2.5_losses_last5'] = 0############# NEW
combined_df_2023['team2_ah+2.5_wins_last5'] = 0############# NEW
combined_df_2023['team2_ah+2.5_losses_last5'] = 0############# NEW

combined_df_2023['team1_over3.5_last5'] = 0############# NEW
combined_df_2023['team1_under3.5_last5'] = 0############# NEW
combined_df_2023['team2_over3.5_last5'] = 0############# NEW
combined_df_2023['team2_under3.5_last5'] = 0############# NEW

combined_df_2023['team1_over4.5_last5'] = 0############# NEW
combined_df_2023['team1_under4.5_last5'] = 0############# NEW
combined_df_2023['team2_over4.5_last5'] = 0############# NEW
combined_df_2023['team2_under4.5_last5'] = 0############# NEW

combined_df_2023['team1_over6.5_last5'] = 0############# NEW
combined_df_2023['team1_under6.5_last5'] = 0############# NEW
combined_df_2023['team2_over6.5_last5'] = 0############# NEW
combined_df_2023['team2_under6.5_last5'] = 0############# NEW


new_cols = ['avg_scr_lasts3_1_home', 'avg_scr_lasts5_1_home', 'avg_scr_lasts3_1_away',
            'avg_scr_lasts5_1_away', 'avg_conc_lasts3_1_home', 'avg_conc_lasts5_1_home',
            'avg_conc_lasts3_1_away', 'avg_conc_lasts5_1_away', 'avg_scr_lasts3_2_home',
            'avg_scr_lasts5_2_home', 'avg_scr_lasts3_2_away', 'avg_scr_lasts5_2_away',
            'avg_conc_lasts3_2_home', 'avg_conc_lasts5_2_home', 'avg_conc_lasts3_2_away',
            'avg_conc_lasts5_2_away','team1_big_wins_last5', 'team1_big_losses_last5', 
            'team2_big_wins_last5', 'team2_big_losses_last5',
            #abaixo vai ser baseado em finalizações
            'avg_total_shots_lasts5_1_home','avg_total_shots_lasts5_1_away','avg_total_shots_lasts5_2_home',
            'avg_total_shots_lasts5_2_away', 'avg_otarget_shots_lasts5_1_home','avg_otarget_shots_lasts5_1_away',
            'avg_otarget_shots_lasts5_2_home','avg_otarget_shots_lasts5_2_away','avg_out_shots_lasts5_1_home',
            'avg_out_shots_lasts5_1_away','avg_out_shots_lasts5_2_home','avg_out_shots_lasts5_2_away',
            'avg_conc_total_shots_lasts5_1_home','avg_conc_total_shots_lasts5_1_away',
            'avg_conc_total_shots_lasts5_2_home','avg_conc_total_shots_lasts5_2_away',
            #abaixo vai ser baseado em corners
            'avg_corners_lasts5_1_home','avg_corners_lasts5_1_away', 
            'avg_corners_conc_lasts5_1_home','avg_corners_conc_lasts5_1_away',
            'avg_corners_lasts5_2_home','avg_corners_lasts5_2_away', 
            'avg_corners_conc_lasts5_2_home', 'avg_corners_conc_lasts5_2_away',
            #abaixo vai ser baseado em fouls
            'avg_fouls_lasts5_1_home','avg_fouls_lasts5_1_away', 
            'avg_fouls_conc_lasts5_1_home', 'avg_fouls_conc_lasts5_1_away',
            'avg_fouls_lasts5_2_home','avg_fouls_lasts5_2_away', 
            'avg_fouls_conc_lasts5_2_home', 'avg_fouls_conc_lasts5_2_away',
            #novas colunas da v7
            'team1_ah-2.5_wins_last5', 'team1_ah-2.5_losses_last5','team2_ah-2.5_wins_last5','team2_ah-2.5_losses_last5',
            'team1_ah+2.5_wins_last5','team1_ah+2.5_losses_last5','team2_ah+2.5_wins_last5','team2_ah+2.5_losses_last5',
            'team1_over3.5_last5','team1_under3.5_last5','team2_over3.5_last5','team2_under3.5_last5',
            'team1_over4.5_last5','team1_under4.5_last5','team2_over4.5_last5','team2_under4.5_last5',
            'team1_over6.5_last5','team1_under6.5_last5','team2_over6.5_last5','team2_under6.5_last5'
            ]

# Create a dictionary with keys as column names and values as np.nan
new_cols_dict = {col: np.nan for col in new_cols}

# Add new columns to the DataFrame
combined_df_2023 = combined_df_2023.assign(**new_cols_dict)


# Iterate over each row in the DataFrame
for i, row in combined_df_2023.iterrows():
    # For each team, get their past home and away matches before the current date
    team1_matches = combined_df_2023[((combined_df_2023['team1'] == row['team1']) | (combined_df_2023['team2'] == row['team1'])) & (combined_df_2023['date'] < row['date']) & (combined_df_2023['season'] == row['season'])].sort_values(by='date')
    team2_matches = combined_df_2023[((combined_df_2023['team1'] == row['team2']) | (combined_df_2023['team2'] == row['team2'])) & (combined_df_2023['date'] < row['date']) & (combined_df_2023['season'] == row['season'])].sort_values(by='date')

    # For each team, calculate stats for last 5 matches
    if not team1_matches.empty:
        team1_matches['big_win'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_big_win'], team1_matches['team2_big_win'])
        team1_matches['big_loss'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_big_loss'], team1_matches['team2_big_loss'])
        combined_df_2023.at[i, 'team1_big_wins_last5'] = team1_matches['big_win'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team1_big_losses_last5'] = team1_matches['big_loss'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan

        # Para ah-2.5 para a equipe 1
        team1_matches['ah-2.5_win'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_ah-2.5_win'], team1_matches['team2_ah-2.5_win'])
        team1_matches['ah-2.5_loss'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_ah-2.5_loss'], team1_matches['team2_ah-2.5_loss'])
        combined_df_2023.at[i, 'team1_ah-2.5_wins_last5'] = team1_matches['ah-2.5_win'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team1_ah-2.5_losses_last5'] = team1_matches['ah-2.5_loss'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan

        # Para ah+2.5 para a equipe 1
        team1_matches['ah+2.5_win'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_ah+2.5_win'], team1_matches['team2_ah+2.5_win'])
        team1_matches['ah+2.5_loss'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_ah+2.5_loss'], team1_matches['team2_ah+2.5_loss'])
        combined_df_2023.at[i, 'team1_ah+2.5_wins_last5'] = team1_matches['ah+2.5_win'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team1_ah+2.5_losses_last5'] = team1_matches['ah+2.5_loss'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan

        # Para over3.5 para a equipe 1
        team1_matches['over3.5'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_over3.5'], team1_matches['team2_over3.5'])
        team1_matches['under3.5'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_under3.5'], team1_matches['team2_under3.5'])
        combined_df_2023.at[i, 'team1_over3.5_last5'] = team1_matches['over3.5'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team1_under3.5_last5'] = team1_matches['under3.5'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan

        # Para over4.5 para a equipe 1
        team1_matches['over4.5'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_over4.5'], team1_matches['team2_over4.5'])
        team1_matches['under4.5'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_under4.5'], team1_matches['team2_under4.5'])
        combined_df_2023.at[i, 'team1_over4.5_last5'] = team1_matches['over4.5'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team1_under4.5_last5'] = team1_matches['under4.5'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan

        # Para over6.5 para a equipe 1
        team1_matches['over6.5'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_over6.5'], team1_matches['team2_over6.5'])
        team1_matches['under6.5'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_under6.5'], team1_matches['team2_under6.5'])
        combined_df_2023.at[i, 'team1_over6.5_last5'] = team1_matches['over6.5'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team1_under6.5_last5'] = team1_matches['under6.5'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan



    if not team2_matches.empty:
        team2_matches['big_win'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_big_win'], team2_matches['team2_big_win'])
        team2_matches['big_loss'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_big_loss'], team2_matches['team2_big_loss'])
        combined_df_2023.at[i, 'team2_big_wins_last5'] = team2_matches['big_win'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team2_big_losses_last5'] = team2_matches['big_loss'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan

        # Para ah-2.5 para a equipe 2
        team2_matches['ah-2.5_win'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_ah-2.5_win'], team2_matches['team2_ah-2.5_win'])
        team2_matches['ah-2.5_loss'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_ah-2.5_loss'], team2_matches['team2_ah-2.5_loss'])
        combined_df_2023.at[i, 'team2_ah-2.5_wins_last5'] = team2_matches['ah-2.5_win'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team2_ah-2.5_losses_last5'] = team2_matches['ah-2.5_loss'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan

        # Para ah+2.5 para a equipe 2
        team2_matches['ah+2.5_win'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_ah+2.5_win'], team2_matches['team2_ah+2.5_win'])
        team2_matches['ah+2.5_loss'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_ah+2.5_loss'], team2_matches['team2_ah+2.5_loss'])
        combined_df_2023.at[i, 'team2_ah+2.5_wins_last5'] = team2_matches['ah+2.5_win'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team2_ah+2.5_losses_last5'] = team2_matches['ah+2.5_loss'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan

        # Para over3.5  para a equipe 2
        team2_matches['over3.5'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_over3.5'], team2_matches['team2_over3.5'])
        team2_matches['under3.5'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_under3.5'], team2_matches['team2_under3.5'])
        combined_df_2023.at[i, 'team2_over3.5_last5'] = team2_matches['over3.5'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team2_under3.5_last5'] = team2_matches['under3.5'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan

        # Para over4.5  para a equipe 2
        team2_matches['over4.5'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_over4.5'], team2_matches['team2_over4.5'])
        team2_matches['under4.5'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_under4.5'], team2_matches['team2_under4.5'])
        combined_df_2023.at[i, 'team2_over4.5_last5'] = team2_matches['over4.5'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team2_under4.5_last5'] = team2_matches['under4.5'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan

        # Para over6.5  para a equipe 2
        team2_matches['over6.5'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_over6.5'], team2_matches['team2_over6.5'])
        team2_matches['under6.5'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_under6.5'], team2_matches['team2_under6.5'])
        combined_df_2023.at[i, 'team2_over6.5_last5'] = team2_matches['over6.5'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team2_under6.5_last5'] = team2_matches['under6.5'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan



    
for i, row in combined_df_2023.iterrows():
    team1_home = combined_df_2023[(combined_df_2023['date'] < row['date']) & (combined_df_2023['team1'] == row['team1']) & (combined_df_2023['season'] == row['season'])]
    team1_away = combined_df_2023[(combined_df_2023['date'] < row['date']) & (combined_df_2023['team2'] == row['team1']) & (combined_df_2023['season'] == row['season'])]
    
    team2_home = combined_df_2023[(combined_df_2023['date'] < row['date']) & (combined_df_2023['team1'] == row['team2']) & (combined_df_2023['season'] == row['season'])]
    team2_away = combined_df_2023[(combined_df_2023['date'] < row['date']) & (combined_df_2023['team2'] == row['team2']) & (combined_df_2023['season'] == row['season'])]

    if not team1_home.empty:
        combined_df_2023.at[i, 'avg_scr_lasts3_1_home'] = team1_home['team1_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team1_home['team1_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_scr_lasts5_1_home'] = team1_home['team1_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team1_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts3_1_home'] = team1_home['team2_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team1_home['team2_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts5_1_home'] = team1_home['team2_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team2_goals'].isna().any() else np.nan
        ####################################### ABAIXO É SOBRE FINALIZAÇÕES ###########################################
        combined_df_2023.at[i, 'avg_total_shots_lasts5_1_home'] = team1_home['team1_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team1_total_shots'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_total_shots_lasts5_1_home'] = team1_home['team2_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team2_total_shots'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_otarget_shots_lasts5_1_home'] = team1_home['team1_shots_on_target'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team1_shots_on_target'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_out_shots_lasts5_1_home'] = team1_home['team1_shots_out'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team1_shots_out'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE CORNERS ##########################################
        combined_df_2023.at[i, 'avg_corners_lasts5_1_home'] = team1_home['team1_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team1_corners'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_corners_conc_lasts5_1_home'] = team1_home['team2_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team2_corners'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE FOULS ##########################################
        combined_df_2023.at[i, 'avg_fouls_lasts5_1_home'] = team1_home['team1_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team1_fouls'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_fouls_conc_lasts5_1_home'] = team1_home['team2_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team2_fouls'].isna().any() else np.nan

    if not team1_away.empty:
        combined_df_2023.at[i, 'avg_scr_lasts3_1_away'] = team1_away['team2_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team1_away['team2_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_scr_lasts5_1_away'] = team1_away['team2_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team2_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts3_1_away'] = team1_away['team1_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team1_away['team1_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts5_1_away'] = team1_away['team1_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team1_goals'].isna().any() else np.nan
        ####################################### ABAIXO É SOBRE FINALIZAÇÕES ###########################################
        combined_df_2023.at[i, 'avg_total_shots_lasts5_1_away'] = team1_away['team2_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team2_total_shots'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_otarget_shots_lasts5_1_away'] = team1_away['team2_shots_on_target'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team2_shots_on_target'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_out_shots_lasts5_1_away'] = team1_away['team2_shots_out'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team2_shots_out'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_total_shots_lasts5_1_away'] = team1_away['team1_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team1_total_shots'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE CORNERS ##########################################
        combined_df_2023.at[i, 'avg_corners_lasts5_1_away'] = team1_away['team2_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team2_corners'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_corners_conc_lasts5_1_away'] = team1_away['team1_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team1_corners'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE FOULS ##########################################
        combined_df_2023.at[i, 'avg_fouls_lasts5_1_away'] = team1_away['team2_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team2_fouls'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_fouls_conc_lasts5_1_away'] = team1_away['team1_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team1_fouls'].isna().any() else np.nan

    if not team2_home.empty:
        combined_df_2023.at[i, 'avg_scr_lasts3_2_home'] = team2_home['team1_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team2_home['team1_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_scr_lasts5_2_home'] = team2_home['team1_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team1_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts3_2_home'] = team2_home['team2_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team2_home['team2_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts5_2_home'] = team2_home['team2_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team2_goals'].isna().any() else np.nan
        ####################################### ABAIXO É SOBRE FINALIZAÇÕES ###########################################
        combined_df_2023.at[i, 'avg_total_shots_lasts5_2_home'] = team2_home['team1_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team1_total_shots'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_otarget_shots_lasts5_2_home'] = team2_home['team1_shots_on_target'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team1_shots_on_target'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_out_shots_lasts5_2_home'] = team2_home['team1_shots_out'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team1_shots_out'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_total_shots_lasts5_2_home'] = team2_home['team2_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team2_total_shots'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE CORNERS ##########################################
        combined_df_2023.at[i, 'avg_corners_lasts5_2_home'] = team2_home['team1_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team1_corners'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_corners_conc_lasts5_2_home'] = team2_home['team2_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team2_corners'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE FOULS ##########################################
        combined_df_2023.at[i, 'avg_fouls_lasts5_2_home'] = team2_home['team1_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team1_fouls'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_fouls_conc_lasts5_2_home'] = team2_home['team2_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team2_fouls'].isna().any() else np.nan


    if not team2_away.empty:
        combined_df_2023.at[i, 'avg_scr_lasts3_2_away'] = team2_away['team2_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team2_away['team2_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_scr_lasts5_2_away'] = team2_away['team2_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team2_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts3_2_away'] = team2_away['team1_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team2_away['team1_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts5_2_away'] = team2_away['team1_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team1_goals'].isna().any() else np.nan
        ####################################### ABAIXO É SOBRE FINALIZAÇÕES ###########################################
        combined_df_2023.at[i, 'avg_total_shots_lasts5_2_away'] = team2_away['team2_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team2_total_shots'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_otarget_shots_lasts5_2_away'] = team2_away['team2_shots_on_target'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team2_shots_on_target'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_out_shots_lasts5_2_away'] = team2_away['team2_shots_out'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team2_shots_out'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_total_shots_lasts5_2_away'] = team2_away['team1_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team1_total_shots'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE CORNERS ##########################################
        combined_df_2023.at[i, 'avg_corners_lasts5_2_away'] = team2_away['team2_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team2_corners'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_corners_conc_lasts5_2_away'] = team2_away['team1_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team1_corners'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE FOULS ##########################################
        combined_df_2023.at[i, 'avg_fouls_lasts5_2_away'] = team2_away['team2_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team2_fouls'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_fouls_conc_lasts5_2_away'] = team2_away['team1_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team1_fouls'].isna().any() else np.nan
        




combined_df_2023.shape        

(1182, 120)

In [ ]:
def get_result(row):
    if row['team1_goals'] > row['team2_goals']:
        return pd.Series([3, 0])
    elif row['team1_goals'] < row['team2_goals']:
        return pd.Series([0, 3])
    else:
        return pd.Series([1, 1])

combined_df_2023[['result_team1', 'result_team2']] = combined_df_2023.apply(get_result, axis=1)

def get_streak(df, result_col, results):
    result_series = df[result_col].apply(lambda x: 1 if x in results else 0)
    result_series = result_series * (result_series.groupby((result_series != result_series.shift()).cumsum()).cumcount() + 1)
    return result_series

# Create a dictionary to hold individual team dataframes
team_df_dict = {}

def get_individual_team_df(df, team_name): #teoricamente aqui deveria ser corners, mas o erro se apresentou menor assim:
    if team_name in team_df_dict:
        return team_df_dict[team_name]
        
    team_games = df[(df['team1'] == team_name) | (df['team2'] == team_name)].copy()
    team_games['team_is_team1'] = team_games['team1'] == team_name
    team_games['team_result'] = np.where(team_games['team_is_team1'], team_games['result_team1'], team_games['result_team2'])
    team_games['team_goals'] = np.where(team_games['team_is_team1'], team_games['team1_goals'], team_games['team2_goals'])
    team_games['team_redcards'] = np.where(team_games['team_is_team1'], team_games['team1_red_cards'], team_games['team2_red_cards'])

    team_games.sort_values('date', inplace=True)
    team_games['days_since_last_game'] = team_games['date'].diff().dt.days

    team_df_dict[team_name] = team_games
    return team_games

def get_team_stats(row, df):
    team1_games = get_individual_team_df(df, row['team1'])
    team2_games = get_individual_team_df(df, row['team2'])

    # Filter to include only games that occurred before the current game
    team1_games = team1_games[team1_games['date'] < row['date']]
    team2_games = team2_games[team2_games['date'] < row['date']]

    stats = {}

    if not team1_games.empty:
        stats['team1_winning_streak'] = get_streak(team1_games, 'team_result', [3]).iloc[-1]
        stats['team1_undefeated_streak'] = get_streak(team1_games, 'team_result', [1, 3]).iloc[-1]
        stats['team1_losing_streak'] = get_streak(team1_games, 'team_result', [0]).iloc[-1]
        stats['team1_without_winning_streak'] = get_streak(team1_games, 'team_result', [0, 1]).iloc[-1]
        stats['avg_points_lasts5_1'] = team1_games.tail(5)['team_result'].mean()
        stats['team1_strength'] = team1_games['team_goals'].sum() / (team1_games['team1_goals'].sum() + team1_games['team2_goals'].sum() + 0.01)
        stats['championship_points_1'] = team1_games['team_result'].sum() / len(team1_games)
        rested_4_or_more_days_1 = team1_games.tail(1)['days_since_last_game'].values[0] >= 4
        stats['rested_4_days_or_more_1'] = 1 if rested_4_or_more_days_1 else -1

    if not team2_games.empty:
        stats['team2_winning_streak'] = get_streak(team2_games, 'team_result', [3]).iloc[-1]
        stats['team2_undefeated_streak'] = get_streak(team2_games, 'team_result', [1, 3]).iloc[-1]
        stats['team2_losing_streak'] = get_streak(team2_games, 'team_result', [0]).iloc[-1]
        stats['team2_without_winning_streak'] = get_streak(team2_games, 'team_result', [0, 1]).iloc[-1]
        stats['avg_points_lasts5_2'] = team2_games.tail(5)['team_result'].mean()
        stats['team2_strength'] = team2_games['team_goals'].sum() / (team2_games['team1_goals'].sum() + team2_games['team2_goals'].sum() + 0.01)
        stats['championship_points_2'] = team2_games['team_result'].sum() / len(team2_games)
        rested_4_or_more_days_2 = team2_games.tail(1)['days_since_last_game'].values[0] >= 4
        stats['rested_4_days_or_more_2'] = 1 if rested_4_or_more_days_2 else -1

    return pd.Series(stats)

combined_df_2023 = pd.concat([combined_df_2023, combined_df_2023.apply(lambda row: get_team_stats(row, combined_df_2023), axis=1)], axis=1)

# Now, calculate the number of suspended players for the next match for each team.
for team_name in team_df_dict.keys():
    team_df = team_df_dict[team_name].copy()
    team_df['next_match_suspended_players'] = team_df['team_redcards'].shift()

    # Assign the suspended players back to the combined_df_2023.
    team1_mask = combined_df_2023['team1'] == team_name
    team2_mask = combined_df_2023['team2'] == team_name
    combined_df_2023.loc[team1_mask, 'team1_suspended_players'] = team_df.loc[team1_mask, 'next_match_suspended_players']
    combined_df_2023.loc[team2_mask, 'team2_suspended_players'] = team_df.loc[team2_mask, 'next_match_suspended_players']

combined_df_2023.shape    


In [6]:
list(combined_df_2023.columns)

['team1',
 'team2',
 'team1_goals',
 'team2_goals',
 'season',
 'championship',
 'team1_shots_on_target',
 'team1_shots_out',
 'team2_shots_on_target',
 'team2_shots_out',
 'team1_red_cards',
 'team2_red_cards',
 'team1_fouls',
 'team2_fouls',
 'team1_corners',
 'team2_corners',
 'team1_total_shots',
 'team2_total_shots',
 'date',
 'is_future_match',
 'goal_diff_team1',
 'goal_diff_team2',
 'corners_diff_team1',
 'corners_diff_team2',
 'team1_big_win',
 'team1_big_loss',
 'team2_big_win',
 'team2_big_loss',
 'team1_ah-2.5_win',
 'team1_ah-2.5_loss',
 'team2_ah-2.5_win',
 'team2_ah-2.5_loss',
 'team1_ah+2.5_win',
 'team1_ah+2.5_loss',
 'team2_ah+2.5_win',
 'team2_ah+2.5_loss',
 'team1_over4.5',
 'team1_under4.5',
 'team2_over4.5',
 'team2_under4.5',
 'team1_over3.5',
 'team1_under3.5',
 'team2_over3.5',
 'team2_under3.5',
 'team1_over6.5',
 'team1_under6.5',
 'team2_over6.5',
 'team2_under6.5',
 'team1_big_wins_last5',
 'team1_big_losses_last5',
 'team2_big_wins_last5',
 'team2_big_loss

In [7]:
columns_to_drop = [
 'team1_goals',
 'team2_goals',
 'team1_shots_on_target',
 'team1_shots_out',
 'team2_shots_on_target',
 'team2_shots_out',
 'team1_red_cards',
 'team2_red_cards',
 'team1_fouls',
 'team2_fouls',
 'team1_corners',
 'team2_corners',
 'team1_total_shots',
 'team2_total_shots',
 'date',
 'is_future_match',
 'goal_diff_team1',
 'goal_diff_team2',
 'corners_diff_team1',
 'corners_diff_team2',
 'team1_big_win',
 'team1_big_loss',
 'team2_big_win',
 'team2_big_loss',
 'team1_ah-2.5_win',
 'team1_ah-2.5_loss',
 'team2_ah-2.5_win',
 'team2_ah-2.5_loss',
 'team1_ah+2.5_win',
 'team1_ah+2.5_loss',
 'team2_ah+2.5_win',
 'team2_ah+2.5_loss',
 'team1_over4.5',
 'team1_under4.5',
 'team2_over4.5',
 'team2_under4.5',
 'team1_over3.5',
 'team1_under3.5',
 'team2_over3.5',
 'team2_under3.5',
 'team1_over6.5',
 'team1_under6.5',
 'team2_over6.5',
 'team2_under6.5',
 'result_team1',
 'result_team2']

Criando future Match baseado na data de hoje

In [8]:
dataset2 = combined_df_2023.copy()

# filter the DataFrame
future_matches = dataset2[dataset2['is_future_match'] == True]
future_matches.sort_values(by='date')
print(future_matches.shape)

from datetime import datetime, timedelta

# Get yesterday's date
yesterday = datetime.now() - timedelta(days=1)

# Filter the DataFrame to include only rows with dates greater than yesterday
#future_matches = future_matches[future_matches['date'] > yesterday]

# Liste todas as colunas que você deseja verificar
columns_to_check= [col for col in future_matches.columns if col not in columns_to_drop]

# Drop as linhas com 'np.nan' nas colunas especificadas
future_matches = future_matches.dropna(subset=columns_to_check)
future_matches_calculado = future_matches.drop(columns_to_drop,axis=1)
future_matches_calculado = future_matches_calculado.drop('season',axis=1)
"""# Contando os valores NaN em cada coluna
nan_counts = future_matches.isna().sum()

# Transformando em uma lista de pares (nome da coluna, contagem de np.nan)
nan_list = list(nan_counts.items())

# Percorrendo a lista e imprimindo cada valor individualmente com o nome da coluna
for col_name, nan_count in nan_list:
    print(f'{col_name}: {nan_count}')"""
    
print(future_matches_calculado.shape)

(443, 140)
(76, 93)


In [9]:
#CORTAR AS LINHAS COM MATCHES FUTUROS AQUI

# Replace empty strings with NaN
print(dataset2.shape)
dataset2.replace('', np.nan, inplace=True)
print(f"A quantidade de np.nan em linhas eram {dataset2.isna().sum().sum()}")
# Remove rows that contain any missing values
dataset2.dropna(inplace=True)

import numpy as np

counter = 0  # Initialize counter
for index, row in dataset2.iterrows():
    if row.isnull().any() or row.eq('').any():
        print(f"Row {index} contains 'NaN' or an empty value.")
        counter += 1  # Increase counter if condition is met

print(f"Total number of rows with 'NaN' or an empty value: {counter}")
dataset2.shape


(1182, 140)
A quantidade de np.nan em linhas eram 47434
Total number of rows with 'NaN' or an empty value: 0


(335, 140)

Concatenando os 2

In [10]:
datatotal = pd.concat([dataset, dataset2], ignore_index=True)#mudei o 'dataset' por combined_df_13c_new

datatotal.sort_values(by='date', inplace=True)

if 'team1_yellow_cards' in datatotal.columns:
    datatotal = datatotal.drop(['team1_yellow_cards'], axis=1)

if 'team2_yellow_cards' in datatotal.columns:
    datatotal = datatotal.drop(['team2_yellow_cards'], axis=1)

# Substituir valores maiores que 15 por 15 na coluna 'team1_corners'
datatotal.loc[datatotal['team1_corners'] > 15, 'team1_corners'] = 15

# Substituir valores maiores que 15 por 15 na coluna 'team2_corners'
datatotal.loc[datatotal['team2_corners'] > 15, 'team2_corners'] = 15    

datatotal

,championship,date,team1,team2,team1_goals,team2_goals,team1_total_shots,team2_total_shots,team1_shots_on_target,team2_shots_on_target,...,team1_winning_streak,team1_without_winning_streak,team2_losing_streak,team2_strength,team2_undefeated_streak,team2_winning_streak,team2_without_winning_streak,team1_suspended_players,team2_suspended_players,is_future_match
0,E0,2002-09-14,Charlton,Arsenal,0.0,3.0,9.0,10.0,3.0,8.0,...,0.0,1.0,0.0,0.666297,5.0,1.0,0.0,0.0,0.0,NaN
1,E0,2002-09-14,Everton,Middlesbrough,2.0,1.0,13.0,10.0,8.0,5.0,...,0.0,3.0,0.0,0.665927,1.0,1.0,0.0,0.0,0.0,NaN
2,E0,2002-09-14,Leeds,Man United,1.0,0.0,8.0,6.0,2.0,5.0,...,1.0,0.0,1.0,0.554939,0.0,0.0,1.0,0.0,0.0,NaN
3,E0,2002-09-14,West Brom,Southampton,1.0,0.0,11.0,10.0,7.0,5.0,...,2.0,0.0,0.0,0.332963,1.0,1.0,0.0,0.0,0.0,NaN
4,E0,2002-09-15,Man City,Blackburn,2.0,2.0,15.0,12.0,7.0,8.0,...,0.0,1.0,2.0,0.454133,0.0,0.0,3.0,1.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67166,ING C,2023-09-09,exeter city,leyton orient,1.0,2.0,5.0,14.0,2.0,7.0,...,3.0,0.0,1.0,0.266489,0.0,0.0,1.0,0.0,0.0,False
67164,ING D,2023-09-09,mk dons,notts county,1.0,1.0,8.0,6.0,3.0,3.0,...,0.0,1.0,0.0,0.545207,5.0,3.0,0.0,0.0,0.0,False
67173,ING D,2023-09-09,wrexham,doncaster rovers,2.0,1.0,12.0,5.0,8.0,2.0,...,1.0,0.0,0.0,0.249844,1.0,0.0,6.0,0.0,1.0,False
67168,ING D,2023-09-09,gillingham,harrogate town fc,1.0,0.0,9.0,4.0,4.0,2.0,...,0.0,2.0,1.0,0.363306,0.0,0.0,1.0,0.0,0.0,False


In [11]:
champ_uniques = datatotal['championship'].unique()
print(champ_uniques)

['E0' 'SC0' 'E3' 'E2' 'E1' 'I1' 'SP1' 'D1' 'F1' 'D2' 'P1' 'SP2' 'T1' 'I2'
 'N1' 'F2' 'B1' 'G1' 'SUE A' 'BRA A' 'BEL A' 'ING C' 'ING D']


Converta Categorias em IDs Numéricos

In [12]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Suponho que 'datatotal' e 'future_matches' já estejam definidos e tenham as mesmas colunas relevantes

# Treinar o LabelEncoder com 'datatotal' e 'future_matches'
le_teams = LabelEncoder().fit(pd.concat([datatotal['team1'], datatotal['team2'], future_matches_calculado['team1'], future_matches_calculado['team2']]).astype(str))
le_champ = LabelEncoder().fit(pd.concat([datatotal['championship'], future_matches_calculado['championship']]).astype(str))

# Aplicar o LabelEncoder a 'datatotal'
datatotal['team1'] = le_teams.transform(datatotal['team1'].astype(str))
datatotal['team2'] = le_teams.transform(datatotal['team2'].astype(str))
datatotal['championship'] = le_champ.transform(datatotal['championship'].astype(str))

# Agora, aplicar o mesmo LabelEncoder a 'future_matches'
future_matches_calculado['team1'] = le_teams.transform(future_matches_calculado['team1'].astype(str))
future_matches_calculado['team2'] = le_teams.transform(future_matches_calculado['team2'].astype(str))
future_matches_calculado['championship'] = le_champ.transform(future_matches_calculado['championship'].astype(str))



In [13]:
future_matches_calculado.head()

,team1,team2,championship,team1_big_wins_last5,team1_big_losses_last5,team2_big_wins_last5,team2_big_losses_last5,team1_ah-2.5_wins_last5,team1_ah-2.5_losses_last5,team2_ah-2.5_wins_last5,...,team2_winning_streak,team2_undefeated_streak,team2_losing_streak,team2_without_winning_streak,avg_points_lasts5_2,team2_strength,championship_points_2,rested_4_days_or_more_2,team1_suspended_players,team2_suspended_players
501,524,475,4,0.0,1.0,1.0,0.0,2.0,3.0,2.0,...,0.0,8.0,0.0,3.0,1.4,0.547067,1.476190,1.0,0.0,1.0
502,550,608,4,0.0,1.0,0.0,1.0,1.0,4.0,3.0,...,0.0,0.0,1.0,6.0,0.6,0.543360,1.333333,1.0,1.0,1.0
503,584,536,4,1.0,1.0,2.0,0.0,2.0,3.0,0.0,...,2.0,2.0,0.0,0.0,1.8,0.576174,1.857143,1.0,0.0,0.0
504,501,477,4,0.0,3.0,2.0,1.0,0.0,5.0,4.0,...,0.0,1.0,0.0,2.0,1.4,0.442223,1.000000,1.0,0.0,0.0
505,528,500,4,2.0,0.0,1.0,0.0,1.0,4.0,1.0,...,0.0,8.0,0.0,3.0,1.4,0.488780,1.238095,1.0,0.0,1.0


In [14]:
n_teams = len(le_teams.classes_)
n_champ = len(le_champ.classes_)
print(n_teams)
print(n_champ)

621
26


In [15]:
datatotal['season'] = datatotal['season'].astype('float64')


Separando X e Y

In [16]:
X = datatotal.drop(columns_to_drop, axis=1)
y1 = datatotal['team1_corners']
y2 = datatotal['team2_corners']
X.shape

(67175, 94)

In [17]:
list(X.dtypes)

[dtype('int32'),
 dtype('int32'),
 dtype('int32'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),

In [18]:
list(X.columns)

['championship',
 'team1',
 'team2',
 'season',
 'team1_big_wins_last5',
 'team1_big_losses_last5',
 'team2_big_wins_last5',
 'team2_big_losses_last5',
 'team1_ah-2.5_wins_last5',
 'team1_ah-2.5_losses_last5',
 'team2_ah-2.5_wins_last5',
 'team2_ah-2.5_losses_last5',
 'team1_ah+2.5_wins_last5',
 'team1_ah+2.5_losses_last5',
 'team2_ah+2.5_wins_last5',
 'team2_ah+2.5_losses_last5',
 'team1_over3.5_last5',
 'team1_under3.5_last5',
 'team2_over3.5_last5',
 'team2_under3.5_last5',
 'team1_over4.5_last5',
 'team1_under4.5_last5',
 'team2_over4.5_last5',
 'team2_under4.5_last5',
 'team1_over6.5_last5',
 'team1_under6.5_last5',
 'team2_over6.5_last5',
 'team2_under6.5_last5',
 'avg_scr_lasts3_1_home',
 'avg_scr_lasts5_1_home',
 'avg_scr_lasts3_1_away',
 'avg_scr_lasts5_1_away',
 'avg_conc_lasts3_1_home',
 'avg_conc_lasts5_1_home',
 'avg_conc_lasts3_1_away',
 'avg_conc_lasts5_1_away',
 'avg_scr_lasts3_2_home',
 'avg_scr_lasts5_2_home',
 'avg_scr_lasts3_2_away',
 'avg_scr_lasts5_2_away',
 'avg_

In [19]:
y1_uniques = datatotal['team1_corners'].unique()
print(y1_uniques)

[ 1. 10.  4.  0. 15.  9.  8.  5.  6.  7.  3. 12. 11. 14.  2. 13.]


In [20]:
# Contar ocorrências para 'y1'
y1_counts = y1.value_counts().sort_index().reindex(range(0, 31), fill_value=0)
print("Contagem de ocorrências para y1:")
print(y1_counts)

# Contar ocorrências para 'y2'
y2_counts = y2.value_counts().sort_index().reindex(range(0, 31), fill_value=0)
print("\nContagem de ocorrências para y2:")
print(y2_counts)


Contagem de ocorrências para y1:
team1_corners
0      695
1     2435
2     5031
3     7584
4     9214
5     9508
6     8631
7     7241
8     5580
9     3953
10    2779
11    1819
12    1115
13     711
14     417
15     462
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
30       0
Name: count, dtype: int64

Contagem de ocorrências para y2:
team2_corners
0      1558
1      4944
2      8157
3     10281
4     10649
5      9482
6      7393
7      5634
8      3583
9      2337
10     1370
11      848
12      415
13      283
14      137
15      104
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        0
30        0
Name: count, dtype: int64


Separando o treino e o teste e a normalização

Algumas variáveis abaixo estão sendo criadas de forma idêntica, apenas para ter significado semântico em relação 

In [21]:
from sklearn.preprocessing import StandardScaler

# Dividindo os dados com base na coluna 'season'
X_train = X[X['season'] < 2022].drop(['team1', 'team2', 'championship', 'season'], axis=1)
X_test = X[X['season'] >= 2022].drop(['team1', 'team2', 'championship', 'season'], axis=1)
y_train1 = y1[X['season'] < 2022]
y_test1 = y1[X['season'] >= 2022]
y_train2 = y2[X['season'] < 2022]
y_test2 = y2[X['season'] >= 2022]

# Escalando apenas as colunas que você quer (ajuste isso conforme suas necessidades)
cols_to_scale = [col for col in future_matches_calculado.columns if col not in ['team1', 'team2', 'championship']]

# Ajustar o escalonador com base no conjunto de treinamento
scaler = StandardScaler().fit(X_train[cols_to_scale])

# Reordenar as colunas para corresponder à ordem usada para ajustar o escalonador
X_train = X_train[cols_to_scale]
X_test = X_test[cols_to_scale]

# Aplicar o escalonamento
future_matches_calculado_scaled = future_matches_calculado.copy()
future_matches_calculado_scaled[cols_to_scale] = scaler.transform(future_matches_calculado[cols_to_scale])

# Transformar os conjuntos de treinamento e teste
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Preparar as colunas para o embedding
X_train_embed = X[X['season'] < 2022][['team1', 'team2', 'championship']]
X_test_embed = X[X['season'] >= 2022][['team1', 'team2', 'championship']]


In [22]:
list(X_train.columns)

['team1_big_wins_last5',
 'team1_big_losses_last5',
 'team2_big_wins_last5',
 'team2_big_losses_last5',
 'team1_ah-2.5_wins_last5',
 'team1_ah-2.5_losses_last5',
 'team2_ah-2.5_wins_last5',
 'team2_ah-2.5_losses_last5',
 'team1_ah+2.5_wins_last5',
 'team1_ah+2.5_losses_last5',
 'team2_ah+2.5_wins_last5',
 'team2_ah+2.5_losses_last5',
 'team1_over3.5_last5',
 'team1_under3.5_last5',
 'team2_over3.5_last5',
 'team2_under3.5_last5',
 'team1_over4.5_last5',
 'team1_under4.5_last5',
 'team2_over4.5_last5',
 'team2_under4.5_last5',
 'team1_over6.5_last5',
 'team1_under6.5_last5',
 'team2_over6.5_last5',
 'team2_under6.5_last5',
 'avg_scr_lasts3_1_home',
 'avg_scr_lasts5_1_home',
 'avg_scr_lasts3_1_away',
 'avg_scr_lasts5_1_away',
 'avg_conc_lasts3_1_home',
 'avg_conc_lasts5_1_home',
 'avg_conc_lasts3_1_away',
 'avg_conc_lasts5_1_away',
 'avg_scr_lasts3_2_home',
 'avg_scr_lasts5_2_home',
 'avg_scr_lasts3_2_away',
 'avg_scr_lasts5_2_away',
 'avg_conc_lasts3_2_home',
 'avg_conc_lasts5_2_home',


In [23]:
print(X_train_scaled.shape)
print(X_test_scaled.shape)
print(y_train1.shape)
print(y_test1.shape)
print(y_train2.shape)
print(y_test2.shape)
print(X_train_embed.shape)
print(X_test_embed.shape)
print(future_matches_calculado_scaled.shape)

(61353, 90)
(5822, 90)
(61353,)
(5822,)
(61353,)
(5822,)
(61353, 3)
(5822, 3)
(76, 93)


Função de perda

In [24]:
"""import tensorflow as tf

def custom_loss(y_true, y_pred):
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])

    avg = tf.reduce_mean(y_true)
    abs_error = tf.abs(y_true - y_pred)
    distance_to_avg = tf.abs(y_pred - avg)
    reward = tf.math.log(distance_to_avg + 3)
    penalty = 2 * abs_error / (distance_to_avg + 3)
    
    # Aplicando a função tanh ao resultado de (penalty - reward)
    normalized_diff = tf.math.tanh(penalty - reward)
    
    # Somando com abs_error
    custom_loss_value = abs_error + normalized_diff
    
    # Garantindo que o valor mínimo da perda seja 0.0001
    custom_loss_value = tf.maximum(custom_loss_value, 0.0001)
    
    return tf.reduce_mean(custom_loss_value)"""

'import tensorflow as tf\n\ndef custom_loss(y_true, y_pred):\n    y_true = tf.reshape(y_true, [-1])\n    y_pred = tf.reshape(y_pred, [-1])\n\n    avg = tf.reduce_mean(y_true)\n    abs_error = tf.abs(y_true - y_pred)\n    distance_to_avg = tf.abs(y_pred - avg)\n    reward = tf.math.log(distance_to_avg + 3)\n    penalty = 2 * abs_error / (distance_to_avg + 3)\n    \n    # Aplicando a função tanh ao resultado de (penalty - reward)\n    normalized_diff = tf.math.tanh(penalty - reward)\n    \n    # Somando com abs_error\n    custom_loss_value = abs_error + normalized_diff\n    \n    # Garantindo que o valor mínimo da perda seja 0.0001\n    custom_loss_value = tf.maximum(custom_loss_value, 0.0001)\n    \n    return tf.reduce_mean(custom_loss_value)'

In [25]:
import tensorflow as tf

def custom_unified_loss(y_true, y_pred):
    # Separar y_true e y_pred para cada time
    y_true_team1, y_true_team2 = y_true[:, 0], y_true[:, 1]
    y_pred_team1, y_pred_team2 = y_pred[:, 0], y_pred[:, 1]
    
    # Calcular a média para cada time
    avg1 = tf.reduce_mean(y_true_team1)
    avg2 = tf.reduce_mean(y_true_team2)

    # Calcular o erro absoluto relativo para cada time
    normalized_error_team1 = tf.abs(y_true[:, 0] - y_pred[:, 0]) 
    normalized_error_team2 = tf.abs(y_true[:, 1] - y_pred[:, 1]) *(avg1/avg2)
    abs_error = (normalized_error_team1 + normalized_error_team2) / 2

    #_val_mae_diff = np.mean(np.abs((val_predict[:, 0] - val_predict[:, 1]) - (val_targ[:, 0] - val_targ[:, 1])))
       # ...
    dif_prev = (y_pred_team1 - y_pred_team2)
    dif_true = (y_true_team1 - y_true_team2)
    
    # Calcula a diferença absoluta entre as diferenças previstas e verdadeiras
    abs_diff = tf.abs(dif_prev - dif_true)
    
    # Calcula um termo de bônus usando a função sigmoidal
    bonus_term = tf.math.tanh(10 * (1 - abs_diff))
    
    # Se a diferença é maior ou igual a 4 (ou menor ou igual a -4), aplica um fator negativo
    condition1 = tf.logical_and(dif_true >= 4, dif_prev >= 4)
    condition2 = tf.logical_and(dif_true <= -4, dif_prev <= -4)
    condition3 = abs_diff <= 1.5

    final_condition = tf.logical_or(tf.logical_or(condition1, condition2), condition3)

    abs_diff_acc = tf.where(final_condition, -1 * (abs_diff + bonus_term), abs_diff + bonus_term)


    # Calcular a diferença entre as previsões para as médias
    distance_to_avg1 = tf.abs(y_pred_team1 - avg1)
    distance_to_avg2 = tf.abs(y_pred_team2 - avg2)
    
    # Calcular os rewards
    reward1 = tf.math.log(distance_to_avg1 + 1)
    reward2 = tf.math.log(distance_to_avg2 + 1)

    # Calcular as penalties
    penalty1 = 1 * normalized_error_team1 / (distance_to_avg1 + 1)
    penalty2 = 1 * normalized_error_team2 / (distance_to_avg2 + 1)    

    # Aplicando a função tanh ao resultado de (penalty - reward)
    normalized_diff1 = tf.math.tanh(penalty1 - reward1)
    normalized_diff2 = tf.math.tanh(penalty2 - reward2)
    normalized_diff = tf.math.tanh((normalized_diff1 + normalized_diff2)/2)

    # combined_error
    combined_error = abs_error + abs_diff_acc + normalized_diff
    
    # Garantindo que o valor mínimo da perda seja 0.0001
    combined_error_value = tf.maximum(combined_error, 0.0001)

    return tf.reduce_mean(combined_error_value)


Testes de funções de perda

In [25]:
import numpy as np
import tensorflow as tf

def custom_unified_loss2(y_true, y_pred):
    # Separar y_true e y_pred para cada time
    y_true_team1, y_true_team2 = y_true[:, 0], y_true[:, 1]
    y_pred_team1, y_pred_team2 = y_pred[:, 0], y_pred[:, 1]
    
    # Calcular a média para cada time
    avg1 = 6.6
    avg2 = 4.6

    # Calcular o erro absoluto relativo para cada time
    normalized_error_team1 = tf.abs(y_true[:, 0] - y_pred[:, 0]) 
    normalized_error_team2 = tf.abs(y_true[:, 1] - y_pred[:, 1]) *(avg1/avg2)
    abs_error = (normalized_error_team1 + normalized_error_team2) / 2

    #_val_mae_diff = np.mean(np.abs((val_predict[:, 0] - val_predict[:, 1]) - (val_targ[:, 0] - val_targ[:, 1])))
       # ...
    dif_prev = (y_pred_team1 - y_pred_team2)
    dif_true = (y_true_team1 - y_true_team2)
    
    # Calcula a diferença absoluta entre as diferenças previstas e verdadeiras
    abs_diff = tf.abs(dif_prev - dif_true)
    
    # Calcula um termo de bônus usando a função sigmoidal
    bonus_term = tf.math.tanh(10 * (1 - abs_diff))
    
    # Se a diferença é maior ou igual a 4 (ou menor ou igual a -4), aplica um fator negativo
    if (dif_true >= 4 and dif_prev >= 4) or (dif_true <= -4 and dif_prev <= -4) or (abs_diff <= 1.5):
        abs_diff_acc = -1 * (abs_diff + bonus_term)
    else:
        abs_diff_acc = abs_diff + bonus_term


    # Calcular a diferença entre as previsões para as médias
    distance_to_avg1 = tf.abs(y_pred_team1 - avg1)
    distance_to_avg2 = tf.abs(y_pred_team2 - avg2)
    
    # Calcular os rewards
    reward1 = tf.math.log(distance_to_avg1 + 1)
    reward2 = tf.math.log(distance_to_avg2 + 1)

    # Calcular as penalties
    penalty1 = 1 * normalized_error_team1 / (distance_to_avg1 + 1)
    penalty2 = 1 * normalized_error_team2 / (distance_to_avg2 + 1)    

    # Aplicando a função tanh ao resultado de (penalty - reward)
    normalized_diff1 = tf.math.tanh(penalty1 - reward1)
    normalized_diff2 = tf.math.tanh(penalty2 - reward2)
    normalized_diff = tf.math.tanh((normalized_diff1 + normalized_diff2)/2)

    # Print rewards, penalties, and normalized_diff
    print(f"reward1: {reward1.numpy()}, penalty1: {penalty1.numpy()}, normalized_diff1: {normalized_diff1.numpy()}")
    print(f"reward2: {reward2.numpy()}, penalty2: {penalty2.numpy()}, normalized_diff2: {normalized_diff2.numpy()}")
    print(f"bonus_term {bonus_term.numpy()}--> abs_diff_acc: {abs_diff_acc.numpy()}, abs_error: {abs_error.numpy()}, normalized_diff: {normalized_diff.numpy()}")  # Novo valor impresso


    # combined_error
    combined_error = abs_error + abs_diff_acc + normalized_diff
    
    # Garantindo que o valor mínimo da perda seja 0.0001
    combined_error_value = tf.maximum(combined_error, 0.0001)

    return tf.reduce_mean(combined_error_value)

# Loop para gerar valores aleatórios e calcular a perda
for _ in range(20):
    # Gerando valores aleatórios para y_true e y_pred
    y_true_sample = np.random.randint(0, 16, size=(1, 2))
    y_pred_sample = np.random.randint(0, 16, size=(1, 2))
    
    # Convertendo para tensores do TensorFlow
    y_true_tensor = tf.convert_to_tensor(y_true_sample, dtype=tf.float32)
    y_pred_tensor = tf.convert_to_tensor(y_pred_sample, dtype=tf.float32)
    
    # Calculando a perda
    loss_value = custom_unified_loss2(y_true_tensor, y_pred_tensor).numpy()
    
    # Imprimindo os resultados
    print(f"y_true_team1: {y_true_sample[0][0]}, y_pred_team1: {y_pred_sample[0][0]}, y_true_team2: {y_true_sample[0][1]}, y_pred_team2: {y_pred_sample[0][1]}, Loss: {loss_value:.4f}\n")


reward1: [0.9555114], penalty1: [0.3846154], normalized_diff1: [-0.5160171]
reward2: [2.00148], penalty2: [2.132785], normalized_diff2: [0.13055553]
bonus_term [-1.]--> abs_diff_acc: [9.], abs_error: [8.391304], normalized_diff: [-0.19037938]
y_true_team1: 4, y_pred_team1: 5, y_true_team2: 0, y_pred_team2: 11, Loss: 17.2009

reward1: [0.9555114], penalty1: [0.3846154], normalized_diff1: [-0.5160171]
reward2: [2.2407095], penalty2: [0.7631823], normalized_diff2: [-0.9010036]
bonus_term [-1.]--> abs_diff_acc: [5.], abs_error: [4.0869565], normalized_diff: [-0.60974187]
y_true_team1: 6, y_pred_team1: 5, y_true_team2: 8, y_pred_team2: 13, Loss: 8.4772

reward1: [1.4816046], penalty1: [2.2727273], normalized_diff1: [0.6590446]
reward2: [2.00148], penalty2: [0.], normalized_diff2: [-0.96413195]
bonus_term [-1.]--> abs_diff_acc: [9.], abs_error: [5.], normalized_diff: [-0.15137136]
y_true_team1: 0, y_pred_team1: 10, y_true_team2: 11, y_pred_team2: 11, Loss: 13.8486

reward1: [2.2407095], pena

Concatenando ambos Ys em um único tensor

In [26]:
import numpy as np

# Suponha que y_train1 e y_train2 são seus vetores de saída originais
# Eles têm o shape (comprimento,)

# Calcular a diferença entre y_train1 e y_train2
y_train_diff = y_train1 - y_train2
y_test_diff = y_test1 - y_test2
# Concatenar ao longo de uma nova dimensão para criar um tensor de saída com shape (comprimento, 3)
y_train_combined = np.stack((y_train1, y_train2, y_train_diff), axis=-1)
y_test_combined = np.stack((y_test1, y_test2, y_test_diff), axis=-1)


print(y_train_combined.shape)
print(y_test_combined.shape)


(61353, 3)
(5822, 3)


Arquitetura da NN

In [27]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam

# ==== Parte 1: Definindo os Inputs para o Embedding ====
# Estes são os inputs que vão alimentar os embeddings. 
# Cada input tem a dimensão de (1,) porque cada jogo tem exatamente um 'team1', um 'team2', e um 'championship'.
team1_input = Input(shape=(1,), name='Team1-Input')
team2_input = Input(shape=(1,), name='Team2-Input')
champ_input = Input(shape=(1,), name='Championship-Input')

# ==== Parte 2: Criando os Embeddings ====
# n_teams e n_champ são o número de times e campeonatos únicos, respectivamente.
# O output_dim é um hiperparâmetro para você ajustar. Ele define o tamanho do espaço de embedding.

# Embedding para o time 1
team1_embedding = Embedding(input_dim=n_teams, output_dim=50, name='Team1-Embedding')(team1_input)  # output_dim ajustável

# Embedding para o time 2
team2_embedding = Embedding(input_dim=n_teams, output_dim=50, name='Team2-Embedding')(team2_input)  # output_dim ajustável

# Embedding para o campeonato
champ_embedding = Embedding(input_dim=n_champ, output_dim=5, name='Championship-Embedding')(champ_input)  # output_dim ajustável

# ==== Parte 3: Achatando os Embeddings ====
# Cada embedding precisa ser achatado para ser concatenado posteriormente
team1_embedding = Flatten()(team1_embedding)
team2_embedding = Flatten()(team2_embedding)
champ_embedding = Flatten()(champ_embedding)

# ==== Parte 4: Outras Características ====
# Este é o input para as outras características (já escaladas) do seu conjunto de dados.
other_features_input = Input(shape=(X_train_scaled.shape[1],), name='Other-Features-Input')

# ==== Parte 5: Concatenando Tudo ====
# Aqui, todos os embeddings e as outras características são concatenados em um único vetor
merged = Concatenate()([team1_embedding, team2_embedding, champ_embedding, other_features_input])

# ==== Parte 6: Camadas Ocultas ====
# Estes são os neurônios e camadas totalmente conectadas (Dense) onde a "aprendizagem" real acontece.
# Você pode ajustar o número de neurônios, a função de ativação, e outros hiperparâmetros aqui.
from tensorflow.keras.regularizers import l1, l2

hidden_layer = Dense(1024, activation='relu', kernel_regularizer=l2(0.0001))(merged)
hidden_layer = Dropout(0.2)(hidden_layer)

hidden_layer = Dense(512, activation='tanh')(hidden_layer)
hidden_layer = Dropout(0.2)(hidden_layer)

hidden_layer = Dense(256, activation='tanh')(hidden_layer)
hidden_layer = Dropout(0.2)(hidden_layer)

hidden_layer = Dense(128, activation='tanh')(hidden_layer)
hidden_layer = Dropout(0.2)(hidden_layer)

hidden_layer = Dense(64, activation='tanh')(hidden_layer)
hidden_layer = Dropout(0.2)(hidden_layer)

hidden_layer = Dense(32, activation='tanh')(hidden_layer)
hidden_layer = Dropout(0.2)(hidden_layer)

#hidden_layer = Dense(8, activation='tanh')(hidden_layer)
#hidden_layer = Dropout(0.2)(hidden_layer)

# ==== Parte 7: Camada de Saída ====
# Esta é a camada de saída. A função de ativação 'linear' é usada para regressão.
output = Dense(3, activation='linear', name='Output-Layer')(hidden_layer)

# ==== Parte 8: Compilando o Modelo ====
# Finalmente, o modelo é compilado. O otimizador Adam é usado, com uma taxa de aprendizagem de 0.001.
# A perda é definida como 'mean_squared_error', que é comum para problemas de regressão.
model = Model(
    inputs=[team1_input, team2_input, champ_input, other_features_input], 
    outputs=[output]
)


In [28]:
from tensorflow.keras.optimizers import Adam
import numpy as np

# Parâmetros para Early Stopping
patience = 10
best_val_mae_diff = np.inf
epochs_without_improvement = 0

# Número de épocas
n_epochs = 100

# Compilando o modelo unificado
model.compile(optimizer=Adam(0.001), loss=custom_unified_loss, metrics=['mean_absolute_error'])

# Loop de treinamento
for epoch in range(n_epochs):
    print(f"Epoch {epoch+1}/{n_epochs}")
    
    # Treinamento para uma época
    history = model.fit(
        [X_train_embed['team1'], X_train_embed['team2'], X_train_embed['championship'], X_train_scaled], 
        y_train_combined, 
        epochs=1,
        batch_size=128,
        verbose=1,
        validation_data=([X_test_embed['team1'], X_test_embed['team2'], X_test_embed['championship'], X_test_scaled], y_test_combined)
    )
    
    # Calculando métricas adicionais
    val_predict = model.predict([X_test_embed['team1'], X_test_embed['team2'], X_test_embed['championship'], X_test_scaled])
    val_targ = y_test_combined

    _val_mae_team1 = np.mean(np.abs(val_predict[:, 0] - val_targ[:, 0]))
    _val_mae_team2 = np.mean(np.abs(val_predict[:, 1] - val_targ[:, 1]))
    _val_mae_diff = np.mean(np.abs((val_predict[:, 0] - val_predict[:, 1]) - (val_targ[:, 0] - val_targ[:, 1])))
    _val_mae_diff_output = np.mean(np.abs(val_predict[:, 2] - val_targ[:, 2]))  # Nova métrica para a terceira saída

    print(f" - val_mae_team1: {_val_mae_team1:.4f} - val_mae_team2: {_val_mae_team2:.4f} - val_mae_diff: {_val_mae_diff:.4f} - val_mae_diff_output: {_val_mae_diff_output:.4f}")

    # Lógica de Early Stopping
    if _val_mae_diff < best_val_mae_diff:
        best_val_mae_diff = _val_mae_diff
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print("Early stopping triggered.")
            break


Epoch 1/100
182/182 [==============================] - 1s 3ms/step
 - val_mae_team1: 2.2409 - val_mae_team2: 1.9768 - val_mae_diff: 3.2370 - val_mae_diff_output: 3.5080
Epoch 2/100
182/182 [==============================] - 0s 2ms/step
 - val_mae_team1: 2.2746 - val_mae_team2: 1.9878 - val_mae_diff: 3.2419 - val_mae_diff_output: 3.6510
Epoch 3/100
182/182 [==============================] - 1s 3ms/step
 - val_mae_team1: 2.2384 - val_mae_team2: 1.9944 - val_mae_diff: 3.2465 - val_mae_diff_output: 3.6113
Epoch 4/100
182/182 [==============================] - 0s 2ms/step
 - val_mae_team1: 2.2389 - val_mae_team2: 2.0420 - val_mae_diff: 3.2987 - val_mae_diff_output: 3.6848
Epoch 5/100
182/182 [==============================] - 1s 3ms/step
 - val_mae_team1: 2.2651 - val_mae_team2: 2.0243 - val_mae_diff: 3.2535 - val_mae_diff_output: 3.5753
Epoch 6/100
182/182 [==============================] - 1s 3ms/step
 - val_mae_team1: 2.2858 - val_mae_team2: 2.0158 - val_mae_diff: 3.2847 - val_mae_diff_o

In [29]:
#2.1901447772979736
#1.996545672416687

In [30]:
future_matches_calculado_scaled.shape

(76, 93)

In [31]:
future_matches_calculado_scaled.head()

,team1,team2,championship,team1_big_wins_last5,team1_big_losses_last5,team2_big_wins_last5,team2_big_losses_last5,team1_ah-2.5_wins_last5,team1_ah-2.5_losses_last5,team2_ah-2.5_wins_last5,...,team2_winning_streak,team2_undefeated_streak,team2_losing_streak,team2_without_winning_streak,avg_points_lasts5_2,team2_strength,championship_points_2,rested_4_days_or_more_2,team1_suspended_players,team2_suspended_players
501,524,475,4,-0.907378,0.123596,0.117516,-0.950273,0.596325,-0.596325,0.504801,...,-0.569612,1.744429,-0.525751,0.436230,0.015667,0.632307,0.374809,0.443706,-0.334677,2.875820
502,550,608,4,-0.907378,0.123596,-0.939662,0.184672,-0.333442,0.333442,1.415019,...,-0.569612,-0.661135,0.419242,1.562600,-1.211908,0.586085,-0.097747,0.443706,2.587813,2.875820
503,584,536,4,0.174966,0.123596,1.174694,-0.950273,0.596325,-0.596325,-1.315635,...,0.986754,-0.059744,-0.525751,-0.690139,0.629454,0.995290,1.634958,0.443706,-0.334677,-0.305174
504,501,477,4,-0.907378,2.344082,1.174694,0.184672,-1.263209,1.263209,2.325237,...,-0.569612,-0.360440,-0.525751,0.060774,0.015667,-0.675169,-1.200378,0.443706,-0.334677,-0.305174
505,528,500,4,1.257310,-0.986648,0.117516,-0.950273,-0.333442,0.333442,-0.405417,...,-0.569612,1.744429,-0.525751,0.436230,0.015667,-0.094563,-0.412784,0.443706,-0.334677,2.875820


Output!!!!!

In [32]:
# Inicializando um dicionário vazio para armazenar as previsões
corner_predictions = {}

# Iterar sobre cada linha em 'future_matches' e 'future_matches_calculado_scaled'
for (index1, row1), (index2, row2) in zip(future_matches.iterrows(), future_matches_calculado_scaled.iterrows()):
    # Prepare os dados de entrada para a previsão
    team1_input_data = np.array([[row2['team1']]], dtype=np.float32)
    team2_input_data = np.array([[row2['team2']]], dtype=np.float32)
    champ_input_data = np.array([[row2['championship']]], dtype=np.float32)
    
    # Certifique-se de que 'other_features_data' contém todas as outras características na mesma ordem que foram usadas para treinar o modelo
    other_features_data = np.array([row2.drop(['team1', 'team2', 'championship']).astype(np.float32)])
    
    # Faça a previsão usando o modelo unificado
    pred = model.predict([team1_input_data, team2_input_data, champ_input_data, other_features_data])
    
    # As previsões para ambos os times estão na mesma saída, então vamos separá-las
    pred_team1, pred_team2 = pred[0][0], pred[0][1]
    
    # Armazenar as previsões no dicionário
    corner_predictions[row1['team1']] = pred_team1
    corner_predictions[row1['team2']] = pred_team2
    
    # Imprimir as previsões
    print(f"Prediction for {row1['team1']}: {pred_team1} corners")
    print(f"Prediction for {row1['team2']}: {pred_team2} corners")

# Exibindo o dicionário de previsões
print("Corner Predictions:", corner_predictions)


1/1 [==============================] - 0s 113ms/step
Prediction for flamengo: 5.931214332580566 corners
Prediction for athletico-pr: 4.1284613609313965 corners
1/1 [==============================] - 0s 24ms/step
Prediction for internacional: 4.3486762046813965 corners
Prediction for são paulo: 5.108304023742676 corners
1/1 [==============================] - 0s 23ms/step
Prediction for red bull bragantino: 6.8913164138793945 corners
Prediction for grêmio: 3.2727508544921875 corners
1/1 [==============================] - 0s 24ms/step
Prediction for coritiba: 4.5252790451049805 corners
Prediction for bahia: 5.442054271697998 corners
1/1 [==============================] - 0s 24ms/step
Prediction for fortaleza: 6.889595031738281 corners
Prediction for corinthians: 3.2711849212646484 corners
1/1 [==============================] - 0s 24ms/step
Prediction for santos: 4.425514221191406 corners
Prediction for cruzeiro: 5.574183940887451 corners
1/1 [==============================] - 0s 24ms/step

In [33]:
# Transformar o dicionário em uma lista de listas, quebrando a cada 2 itens
items = list(corner_predictions.items())
rows = [items[i:i + 2] for i in range(0, len(items), 2)]

# Criar um DataFrame a partir da lista de listas
df = pd.DataFrame(rows, columns=['Team1', 'Team2'])

# Separar as tuplas em duas colunas separadas para os nomes das equipes e as previsões
df[['Team1_name', 'Team1_prediction']] = pd.DataFrame(df['Team1'].tolist(), index=df.index)
df[['Team2_name', 'Team2_prediction']] = pd.DataFrame(df['Team2'].tolist(), index=df.index)

# Descartar as colunas originais e reordenar
df = df[['Team1_name', 'Team1_prediction', 'Team2_name', 'Team2_prediction']]


df

,Team1_name,Team1_prediction,Team2_name,Team2_prediction
0,flamengo,5.931214,athletico-pr,4.128461
1,internacional,4.348676,são paulo,5.108304
2,red bull bragantino,6.891316,grêmio,3.272751
3,coritiba,4.525279,bahia,5.442054
4,fortaleza,6.889595,corinthians,3.271185
...,...,...,...,...
71,elfsborg,6.891590,kalmar,3.271631
72,spvgg unterhaching,5.354463,fc saarbrucken,4.236595
73,aik,4.870977,degerfors if,2.553196
74,norrkoping,4.241297,mjallby,2.910828


In [34]:
df.shape

(76, 4)

In [35]:
#STEP 2

from scipy.stats import poisson
import numpy as np
import pandas as pd

def poisson_probabilities(lam, max_corners=15):
    probs = [poisson.pmf(k, lam) for k in range(max_corners + 1)]
    probs.append(1 - sum(probs))
    return probs

def calculate_probabilities(team1_corners_prediction, team2_corners_prediction):
    team1_corners_probabilities = poisson_probabilities(team1_corners_prediction)
    team2_corners_probabilities = poisson_probabilities(team2_corners_prediction)

    joint_prob_matrix = np.outer(team1_corners_probabilities, team2_corners_probabilities)

    team1_win_prob = np.sum(np.tril(joint_prob_matrix, -1))
    draw_prob = np.sum(np.diag(joint_prob_matrix))
    team2_win_prob = np.sum(np.triu(joint_prob_matrix, 1))

    team1_minus35_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if i-j>=4])
    team1_plus35_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if i-j>=-3])
    team2_minus35_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if j-i>=4])
    team2_plus35_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if j-i>=-3])

    team1_minus25_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if i-j>=3])
    team1_plus25_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if i-j>=-2])
    team2_minus25_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if j-i>=3])
    team2_plus25_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if j-i>=-2])

    team1_minus15_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if i-j>=2])
    team1_plus15_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if i-j>=-1])
    team2_minus15_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if j-i>=2])
    team2_plus15_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if j-i>=-1])

    team1_over45_prob = 1 - sum(team1_corners_probabilities[:5])
    team1_under45_prob = sum(team1_corners_probabilities[:5])
    team2_over45_prob = 1 - sum(team2_corners_probabilities[:5])
    team2_under45_prob = sum(team2_corners_probabilities[:5])

    team1_over55_prob = 1 - sum(team1_corners_probabilities[:6])
    team1_under55_prob = sum(team1_corners_probabilities[:6])
    team2_over55_prob = 1 - sum(team2_corners_probabilities[:6])
    team2_under55_prob = sum(team2_corners_probabilities[:6])

    team1_over65_prob = 1 - sum(team1_corners_probabilities[:7])
    team1_under65_prob = sum(team1_corners_probabilities[:7])
    team2_over65_prob = 1 - sum(team2_corners_probabilities[:7])
    team2_under65_prob = sum(team2_corners_probabilities[:7])
    
    return (team1_win_prob, draw_prob, team2_win_prob,
        team1_minus15_prob, team1_plus15_prob, team2_minus15_prob, team2_plus15_prob,
        team1_minus25_prob, team1_plus25_prob, team2_minus25_prob, team2_plus25_prob,
        team1_minus35_prob, team1_plus35_prob, team2_minus35_prob, team2_plus35_prob,
        team1_over45_prob, team1_under45_prob, team2_over45_prob, team2_under45_prob,
        team1_over55_prob, team1_under55_prob, team2_over55_prob, team2_under55_prob,
        team1_over65_prob, team1_under65_prob, team2_over65_prob, team2_under65_prob)





In [36]:
# Inicializando o DataFrame para armazenar as odds
odds_df = pd.DataFrame(columns=['Team 1', 'Team 2', 'Team 1 Win Odd', 'Draw Odd', 'Team 2 Win Odd',
                                'Team 1 -1.5 Odd', 'Team 1 +1.5 Odd', 'Team 2 -1.5 Odd', 'Team 2 +1.5 Odd',
                                'Team 1 -2.5 Odd', 'Team 1 +2.5 Odd', 'Team 2 -2.5 Odd', 'Team 2 +2.5 Odd',
                                'Team 1 -3.5 Odd', 'Team 1 +3.5 Odd', 'Team 2 -3.5 Odd', 'Team 2 +3.5 Odd',
                                'Team 1 Over 4.5', 'Team 1 Under 4.5', 'Team 2 Over 4.5', 'Team 2 Under 4.5',
                                'Team 1 Over 5.5', 'Team 1 Under 5.5', 'Team 2 Over 5.5', 'Team 2 Under 5.5',
                                'Team 1 Over 6.5', 'Team 1 Under 6.5', 'Team 2 Over 6.5', 'Team 2 Under 6.5'])

# Iterar sobre cada linha no DataFrame 'df'
for index, row in df.iterrows():
    team1_name = row['Team1_name']
    team1_prediction = row['Team1_prediction']
    team2_name = row['Team2_name']
    team2_prediction = row['Team2_prediction']
    
    # Calculando as probabilidades usando a função do STEP 2
    probabilities = calculate_probabilities(team1_prediction, team2_prediction)
    
    # Calculando as odds
    odds = [1 / prob for prob in probabilities]
    
    # Adicionando as odds ao DataFrame
    odds_df.loc[len(odds_df)] = [team1_name, team2_name] + odds

    # Inicializando as novas colunas
odds_df['date'] = None
odds_df['championship'] = None

# Preenchendo as novas colunas
for index, row in odds_df.iterrows():
    team1 = row['Team 1']
    team2 = row['Team 2']
    
    # Encontrando a linha correspondente em 'future_matches'
    match_row = future_matches[(future_matches['team1'] == team1) & (future_matches['team2'] == team2)]
    
    if not match_row.empty:
        # Se encontrarmos uma linha correspondente, atualizamos 'odds_df'
        odds_df.at[index, 'date'] = match_row.iloc[0]['date']
        odds_df.at[index, 'championship'] = match_row.iloc[0]['championship']

# Converte a coluna 'date' para o tipo de data do pandas
odds_df['date'] = pd.to_datetime(odds_df['date'])

# Formata a coluna 'date' para o formato de data brasileiro (dd/mm/yyyy)
odds_df['date'] = odds_df['date'].dt.strftime('%d/%m/%Y')

odds_df = odds_df.sort_values(['championship', 'date'])

# Lista das colunas atuais
cols = odds_df.columns.tolist()

# Removendo 'date' e 'championship' da lista
cols.remove('date')
cols.remove('championship')

# Reordenando as colunas
new_cols = ['date', 'championship'] + cols

# Atualizando o DataFrame com a nova ordem de colunas
odds_df = odds_df[new_cols]

odds_df.to_excel("output_corners_NNNN_AH(withoutG).xlsx", index=False)

# Exibindo o DataFrame de odds
print(odds_df.shape)


(76, 31)


Criando e tratando os times para verificar jogos próximos

In [37]:
lista_teams = list(odds_df['Team1_name']) + list(odds_df['Team2_name']) 

In [38]:
lista_teams

['flamengo',
 'internacional',
 'red bull bragantino',
 'coritiba',
 'fortaleza',
 'santos',
 'fc nuremberga',
 'sc paderborn',
 'salford city',
 'real saragoza',
 'palmeiras',
 'cuiabá',
 'kvc westerlo',
 'msv duisburgo',
 'bradford city',
 'vasco da gama',
 'albacete',
 'real valladolid',
 'forest green rovers',
 'accrington stanley',
 'afc wimbledon',
 'fleetwood town',
 'huesca',
 'gillingham',
 'wrexham',
 'stevenage f.c.',
 'reading',
 'shrewsbury town',
 'newport county afc',
 'mk dons',
 'atlético mineiro',
 'crawley town',
 'port vale fc',
 'rot-weiss essen',
 'peterborough united',
 'karlsruher sc',
 'wycombe wanderers',
 'derby county',
 'barnsley fc',
 'lincoln city',
 'clube brugge',
 'rwd molenbeek 47',
 'royal union saint-gilloise',
 'hansa rostock',
 'ifk gotemburgo',
 'exeter city',
 'djurgardens if',
 'sv sandhausen',
 'sv elversberg',
 'vfb lubeck',
 'bielefeld',
 'schalke 04',
 'ik sirius',
 'borussia dortmund ii',
 'fc ingolstadt',
 'wigan fc',
 'eibar',
 'hertha b

Printando as diferenças de datas

In [39]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
from datetime import datetime

# Dicionário para converter o nome do mês de português para inglês
month_map = {
    'jan': 'Jan',
    'fev': 'Feb',
    'mar': 'Mar',
    'abr': 'Apr',
    'mai': 'May',
    'jun': 'Jun',
    'jul': 'Jul',
    'ago': 'Aug',
    'set': 'Sep',
    'out': 'Oct',
    'nov': 'Nov',
    'dez': 'Dec'
}

# Função para extrair a data do texto do confronto
def extract_date(confronto_text):
    date_match = re.search(r'(\d{1,2}) (set|out|jan|fev|mar|abr|mai|jun|jul|ago|nov|dez)', confronto_text.lower())
    if date_match:
        day = date_match.group(1)
        month_pt = date_match.group(2)
        month_en = month_map.get(month_pt)
        date_str = f"{day} {month_en}"
        return datetime.strptime(date_str, '%d %b')
    return None

# Restante do código segue o mesmo.


def is_relevant_game(confronto_text):
    irrelevant_keywords = ['Femin', 'Sub', 'women', 'u20']
    for keyword in irrelevant_keywords:
        if keyword.lower() in confronto_text.lower():
            return False
    return True

def colored_print(text, days):
    color_code = "\033[91m" if days <= 4 else "\033[0m"  # 91 é o código para vermelho
    print(f"{text} {color_code}{days}\033[0m")  # \033[0m reseta a cor

with open('saida_dif_dias.txt', 'w') as f:
    for time in lista_teams:
        try:
            driver = webdriver.Chrome()
            driver.maximize_window()
            driver.get('https://oddspedia.com/br/futebol')
            
            wait = WebDriverWait(driver, 10)
            search_box = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__layout"]/div/div[1]/div[2]/div[2]/aside/div[1]/div/input')))
            
            search_box.send_keys(time)
            search_box.send_keys(Keys.RETURN)
            
            all_matches = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="__layout"]/div/div[1]/div[2]/div[2]/aside/div[1]/div[2]/ul/li/div')))[:7]
            
            filtered_matches = [match for match in all_matches if is_relevant_game(match.text)]
            
            filtered_dates = [extract_date(match.text) for match in filtered_matches if extract_date(match.text)]

            f.write(f"\nPróximas partidas para {time}:\n")
            print(f"\nPróximas partidas para {time}:")
            
            for i, match in enumerate(filtered_matches):
                output = f"\nJogo {i+1}:\n{match.text}\n"  # Conteúdo do jogo
                f.write(output)  # Escreve no arquivo
                print(output)  # Imprime na tela
                
                if i < len(filtered_dates) - 1:
                    diff = (filtered_dates[i + 1] - filtered_dates[i]).days
                    colored_output = f"Diferença de dias para o próximo jogo: {diff}\n"
                    f.write(colored_output)  # Escreve no arquivo
                    colored_print("Diferença de dias para o próximo jogo:", diff)  # Imprime na tela
            
            f.write("-"*70 + "\n")  # Escreve no arquivo
            print("-"*70)
            
        except Exception as e:
            error_output = f"Erro: {str(e)}\n"
            f.write(error_output)  # Escreve no arquivo
            print(error_output)  # Imprime na tela
            import traceback
            traceback.print_exc()
        
        finally:
            driver.close()









Próximas partidas para flamengo:

Jogo 1:
BRASILEIRÃO SÉRIE A
13 set
Flamengo
Athletico-PR

Diferença de dias para o próximo jogo: 4

Jogo 2:
COPA DO BRASIL
17 set
Flamengo
São Paulo

Diferença de dias para o próximo jogo: 3

Jogo 3:
BRASILEIRÃO SÉRIE A
20 set
Goiás
Flamengo

Diferença de dias para o próximo jogo: 4

Jogo 4:
COPA DO BRASIL
24 set
São Paulo
Flamengo

Diferença de dias para o próximo jogo: 7

Jogo 5:
BRASILEIRÃO SÉRIE A
01 out
Flamengo
Bahia

----------------------------------------------------------------------

Próximas partidas para internacional:

Jogo 1:
LIGA DRUGA CRNE GORE
13 set
FK Lovcen
FK Internacional Podgorica

Diferença de dias para o próximo jogo: 0

Jogo 2:
BRASILEIRÃO SÉRIE A
13 set
Internacional
São Paulo

Diferença de dias para o próximo jogo: 3

Jogo 3:
SINGAPURA PRIMEIRA LIGA
16 set
Home United
Geylang Internacional

Diferença de dias para o próximo jogo: 1

Jogo 4:
LIGA DRUGA CRNE GORE
17 set
FK Internacional Podgorica
FK Iskra Danilovgrad

Diferen

Traceback (most recent call last):
  File "C:\Users\mathe\AppData\Local\Temp\ipykernel_3088\3735419640.py", line 63, in <module>
    all_matches = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="__layout"]/div/div[1]/div[2]/div[2]/aside/div[1]/div[2]/ul/li/div')))[:7]
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mathe\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\support\wait.py", line 95, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 




Próximas partidas para fc nuremberga:

Jogo 1:
LIGA REGIONAL BAYERN
15 set
Greuther Furth II
1 FC Nuremberga II

Diferença de dias para o próximo jogo: 0

Jogo 2:
2ª BUNDESLIGA
15 set
FC Nuremberga
Greuther Furth

Diferença de dias para o próximo jogo: 8

Jogo 3:
2ª BUNDESLIGA
23 set
Eintracht Braunschweig
FC Nuremberga

Diferença de dias para o próximo jogo: 0

Jogo 4:
LIGA REGIONAL BAYERN
23 set
1 FC Nuremberga II
Eintracht Bamberg 2010

----------------------------------------------------------------------

Próximas partidas para sc paderborn:

Jogo 1:
2ª BUNDESLIGA
15 set
SC Paderborn
Wehen Wiesbaden

Diferença de dias para o próximo jogo: 7

Jogo 2:
2ª BUNDESLIGA
22 set
FC Magdeburg
SC Paderborn

Diferença de dias para o próximo jogo: 0

Jogo 3:
REGIONALLIGA OESTE
22 set
Monchengladbach II
SC Paderborn 07 II

----------------------------------------------------------------------

Próximas partidas para salford city:

Jogo 1:
LEAGUE TWO
15 set
Salford City
Notts County

Diferença 

Traceback (most recent call last):
  File "C:\Users\mathe\AppData\Local\Temp\ipykernel_3088\3735419640.py", line 63, in <module>
    all_matches = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="__layout"]/div/div[1]/div[2]/div[2]/aside/div[1]/div[2]/ul/li/div')))[:7]
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mathe\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\support\wait.py", line 95, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 




Próximas partidas para kvc westerlo:

Jogo 1:
PRIMEIRA DIVISÃO A
15 set
KVC Westerlo
Royal Antwerp FC

Diferença de dias para o próximo jogo: 7

Jogo 2:
PRIMEIRA DIVISÃO A
22 set
Standard Liege
KVC Westerlo

Diferença de dias para o próximo jogo: 9

Jogo 3:
PRIMEIRA DIVISÃO A
01 out
Genk
KVC Westerlo

----------------------------------------------------------------------

Próximas partidas para msv duisburgo:

Jogo 1:
3. LIGA
15 set
MSV Duisburgo
SC Verl

Diferença de dias para o próximo jogo: 8

Jogo 2:
3. LIGA
23 set
Köln
MSV Duisburgo

Diferença de dias para o próximo jogo: 4

Jogo 3:
3. LIGA
27 set
Borussia Dortmund II
MSV Duisburgo

Diferença de dias para o próximo jogo: 3

Jogo 4:
3. LIGA
30 set
MSV Duisburgo
Preusen Munster

----------------------------------------------------------------------

Próximas partidas para bradford city:

Jogo 1:
LEAGUE TWO
16 set
Bradford City
Harrogate Town FC

Diferença de dias para o próximo jogo: 7

Jogo 2:
LEAGUE TWO
23 set
Newport County AFC


...


Próximas partidas para osnabruck:

Jogo 1:
2ª BUNDESLIGA
17 set
Hannover Sportverein
Osnabruck

Diferença de dias para o próximo jogo: 5

Jogo 2:
2ª BUNDESLIGA
22 set
Osnabruck
Hamburgo SV

----------------------------------------------------------------------

Próximas partidas para standard liege:

Jogo 1:
PRIMEIRA DIVISÃO A
17 set
Eupen
Standard Liege

Diferença de dias para o próximo jogo: 5

Jogo 2:
PRIMEIRA DIVISÃO A
22 set
Standard Liege
KVC Westerlo

Diferença de dias para o próximo jogo: 8

Jogo 3:
PRIMEIRA DIVISÃO A
30 set
Oud-Heverlee Leuven
Standard Liege

Diferença de dias para o próximo jogo: 8

Jogo 4:
PRIMEIRA DIVISÃO A
08 out
Standard Liege
Clube Brugge

Diferença de dias para o próximo jogo: 13

Jogo 5:
PRIMEIRA DIVISÃO A
21 out
Standard Liege
Anderlecht

----------------------------------------------------------------------
Erro: Message: 




Traceback (most recent call last):
  File "C:\Users\mathe\AppData\Local\Temp\ipykernel_3088\3735419640.py", line 63, in <module>
    all_matches = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="__layout"]/div/div[1]/div[2]/div[2]/aside/div[1]/div[2]/ul/li/div')))[:7]
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mathe\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\support\wait.py", line 95, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 




Próximas partidas para anderlecht:

Jogo 1:
PRIMEIRA DIVISÃO A
17 set
KV Kortrijk
Anderlecht

Diferença de dias para o próximo jogo: 7

Jogo 2:
PRIMEIRA DIVISÃO A
24 set
Anderlecht
Clube Brugge

Diferença de dias para o próximo jogo: 6

Jogo 3:
PRIMEIRA DIVISÃO A
30 set
Eupen
Anderlecht

----------------------------------------------------------------------
Erro: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7A5BB52A2+57122]
	(No symbol) [0x00007FF7A5B2EA92]
	(No symbol) [0x00007FF7A59FE3AB]
	(No symbol) [0x00007FF7A5A37D3E]
	(No symbol) [0x00007FF7A5A37E2C]
	(No symbol) [0x00007FF7A5A70B67]
	(No symbol) [0x00007FF7A5A5701F]
	(No symbol) [0x00007FF7A5A6EB82]
	(No symbol) [0x00007FF7A5A56DB3]
	(No symbol) [0x00007FF7A5A2D2B1]
	(No symbol) [0x00007FF7A5A2E494]
	GetHandleVerifier [0x00007FF7A5E5EF82+2849794]
	GetHandleVerifier [0x00007FF7A5EB1D24+3189156]
	GetHandleVerifier [0x00007FF7A5EAACAF+3160367]
	GetHandleVerifier [0x00007FF7A5C46D06+653702]
	(No symbol) [0x00007FF7A5B3A208]


Traceback (most recent call last):
  File "C:\Users\mathe\AppData\Local\Temp\ipykernel_3088\3735419640.py", line 58, in <module>
    search_box = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__layout"]/div/div[1]/div[2]/div[2]/aside/div[1]/div/input')))
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mathe\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\support\wait.py", line 95, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7A5BB52A2+57122]
	(No symbol) [0x00007FF7A5B2EA92]
	(No symbol) [0x00007FF7A59FE3AB]
	(No symbol) [0x00007FF7A5A37D3E]
	(No symbol) [0x00007FF7A5A37E2C]
	(No symbol) [0x00007FF7A5A70B67]
	(No symbol) [0x00007FF7A5A5701F]
	(No symbol) [0x00007FF7A5A6EB82]
	(No symbol) [0x00007FF7A5A56DB3]
	(No sym


Próximas partidas para halmstad:

Jogo 1:
LIGA ALLSVENSKAN
17 set
BK Hacken
Halmstad

Diferença de dias para o próximo jogo: 8

Jogo 2:
LIGA ALLSVENSKAN
25 set
Halmstad
Elfsborg

Diferença de dias para o próximo jogo: 5

Jogo 3:
LIGA ALLSVENSKAN
30 set
Djurgardens IF
Halmstad

----------------------------------------------------------------------

Próximas partidas para holstein kiel:

Jogo 1:
LIGA REGIONAL NORTE
13 set
Holstein Kiel II
Meppen

Diferença de dias para o próximo jogo: 4

Jogo 2:
LIGA REGIONAL NORTE
17 set
Holstein Kiel II
SC Spelle-Venhaus

Diferença de dias para o próximo jogo: 0

Jogo 3:
2ª BUNDESLIGA
17 set
FC St. Pauli
Holstein Kiel

Diferença de dias para o próximo jogo: 6

Jogo 4:
LIGA REGIONAL NORTE
23 set
Holstein Kiel II
Bremer SV

Diferença de dias para o próximo jogo: 1

Jogo 5:
2ª BUNDESLIGA
24 set
Holstein Kiel
Hertha Berlim

----------------------------------------------------------------------

Próximas partidas para ssv ulm 1846:

Jogo 1:
3. LIGA
17 set
